# Linked Data with Omeka S Provided Example

Linked data can be retrieved from the Omeka S API,
and you can process this information using the `rdflib` in Python.
In short, this notebook illustrates an extract and transform process
that retrieves two data sources (i.e., *extracts* data from existing
Omeka S items and an imaginary triple store), then combines
the information in a new data graph that can be expressed in
triples (i.e., *transformed* into a new data structure).

This notebook continues from `omeka-s-linked-data-demos.ipynb`, which undertook the steps above.
In this next step, the goal is to transform the data back into JSON-LD,
then return it to Omeka S (i.e., *load*).

## Setup

This section imports requests for working the REST API. More importantly, however,
are various import statements for `rdflib`.
Of particular note are the import of various specific RDF datatypes, including Graph, URIRef, Literal, BNode (for blank nodes), and Namespace.
The serializer and parser functions assist in processing graph data into various transport formats, including RDF in XML, JSON, and turtle, among others.
Finally, note the last import line, which imports various built-in schemes that can be used as namespaces,
including regular Resource Description Framework datatypes (both RDF and RDFs), Friend of a Friend (FOAF), DublinCore extended terms (DCTERMS), and schema.org (SDO).

In [1]:
import requests
import json
import rdflib
from rdflib import Graph, URIRef, Literal, BNode, Namespace, plugin, Variable
from rdflib.serializer import Serializer
from rdflib.plugin import register, Parser
from rdflib.namespace import RDF, RDFS, FOAF, DCTERMS, SDO

In [2]:
# create sample data to add to the graph
newData = {
    'Jane Austen' : {
        'https://schema.org/deathDate' : 1817,
        'https://schema.org/birthDate' : 1775,
        'https://schema.org/deathPlace': 'https://en.wikipedia.org/wiki/England'
    },
    'Octavia E. Butler' : { 
        'https://schema.org/deathDate' : 2006,
        'https://schema.org/birthDate' : 1947,
        'https://schema.org/deathPlace': 'https://en.wikipedia.org/wiki/Lake_Forest_Park,_Washington'
        },
    'Herman Melville' : { 
        'https://schema.org/deathDate' : 1891,
        'https://schema.org/birthDate' : 1819,
        'https://schema.org/deathPlace' : 'https://en.wikipedia.org/wiki/New_York_City'
        }
}

Add namespace information for Omeka S's scheme:

In [3]:
omekas_ns = Namespace('http://omeka.org/s/vocabs/o#')

## Retrieve Data from Omeka S

Search for all the items in the specified set

In [4]:
url = 'http://jajohnst.si676.si.umich.edu/omeka-s/api'

action = '/items'

# if you create items in your Omeka S site,
# your item set will have a different id (specific to your site)
parameters = {
    'item_set_id':311,
}

In [5]:
r = requests.get(url + action, params=parameters)

print(r.url)
print(r.status_code)

http://jajohnst.si676.si.umich.edu/omeka-s/api/items?item_set_id=311
200


In [6]:
r.json()

[{'@context': 'http://jajohnst.si676.si.umich.edu/omeka-s/api-context',
  '@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/items/303',
  '@type': 'o:Item',
  'o:id': 303,
  'o:is_public': True,
  'o:owner': {'@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/users/2',
   'o:id': 2},
  'o:resource_class': None,
  'o:resource_template': None,
  'o:thumbnail': None,
  'o:title': 'A Mere Title for an item created via the API',
  'thumbnail_display_urls': {'large': None, 'medium': None, 'square': None},
  'o:created': {'@value': '2025-11-07T23:16:45+00:00',
   '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
  'o:modified': {'@value': '2025-11-14T04:56:54+00:00',
   '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
  'o:primary_media': None,
  'o:media': [],
  'o:item_set': [{'@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/item_sets/195',
    'o:id': 195},
   {'@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/item_sets/311',
    'o:id': 311}],
  'o:site': [{

For the later *load* step, which will not be illustrated in this notebook,
create a set list of the Omeka S item IDs. This would be used as a feeder
list to load items back using the API, which uses the item ID.

In [7]:
set = r.json()

print(f'Set has {len(set)} items')

Set has 6 items


In [8]:
for item in set:
    print(item['o:id'])

303
306
308
309
310
312


Now, store those IDs in a list

In [9]:
set_list = []

for item in r.json():
    set_list.append(item['o:id'])

print(set_list)

[303, 306, 308, 309, 310, 312]


## Parse data with the RDFLib module

Using the `rdflib` module capabilities, parse this data.

First, create an RDF graph from it. Note in the following the usage of the `bind_namespaces` argument, which binds all of the imported namespaces
so that they are available later when writing new triples in the graph.

In [10]:
g = Graph(bind_namespaces='rdflib').parse(data=r.text, format='json-ld')

Add the Omeka S namespace (`omekas_ns`, prefixed as `o`):

In [11]:
g.bind('o',omekas_ns)

Now, look through the graph. The graph is a series of "triples",
which are subject-predicate-object tuples. These can be modified. example, after the initial look, you can remove all of those with the Omeka S namespace (`o`).
Note that RDFLib may drop or delete any orphaned subjects or objects that may not be part of a triple. 

In [12]:
for s, p, o in g:
    print(f'{s} -> {p} -> {o} .')

http://jajohnst.si676.si.umich.edu/omeka-s/api/items/308 -> http://omeka.org/s/vocabs/o#resource_class -> http://jajohnst.si676.si.umich.edu/omeka-s/api/resource_classes/1 .
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/306 -> http://omeka.org/s/vocabs/o#is_public -> true .
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/303 -> http://omeka.org/s/vocabs/o#created -> 2025-11-07T23:16:45+00:00 .
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/308 -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://purl.org/dc/terms/Agent .
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/310 -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://purl.org/dc/terms/Agent .
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/310 -> http://omeka.org/s/vocabs/o#id -> 310 .
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/308 -> http://omeka.org/s/vocabs/o#created -> 2025-11-14T03:35:42+00:00 .
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/306 -> http://omeka.org

### Outputting, saving, and serializing

Convert the graph to the terse triple format, a.k.a. *Turtle*:

In [13]:
ser = g.serialize(format='turtle')

print(ser)

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix o: <http://omeka.org/s/vocabs/o#> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://jajohnst.si676.si.umich.edu/omeka-s/api/items/303> a o:Item ;
    o:created "2025-11-07T23:16:45+00:00"^^xsd:dateTime ;
    o:id 303 ;
    o:is_public true ;
    o:item_set <http://jajohnst.si676.si.umich.edu/omeka-s/api/item_sets/195>,
        <http://jajohnst.si676.si.umich.edu/omeka-s/api/item_sets/311> ;
    o:modified "2025-11-14T04:56:54+00:00"^^xsd:dateTime ;
    o:owner <http://jajohnst.si676.si.umich.edu/omeka-s/api/users/2> ;
    o:site <http://jajohnst.si676.si.umich.edu/omeka-s/api/sites/1> ;
    o:title "A Mere Title for an item created via the API" ;
    dcterms:rights "No known restrictions on publication."@en-us ;
    dcterms:title "A Mere Title for an item created via the API"@en-us .

<http://jajohnst.si676.si.umich.edu/omeka-s/api/item

Save it to a file

In [14]:
with open('item-set-graph-1.ttl', 'w') as f:
    f.write(ser)

## Parsing, Modifying, and Adding to the Graph

Now try to remove the Omeka data in order to get a closer look
at the collection specific data.
To do this, the following loop uses `.remove()` to delete triples
with the Omeka S vocabulary namespace in the *predicate* (element 1 of the tuple).

In [15]:
# remove the omeka specific data
for triple in g:
    if 'http://omeka.org/s/vocabs/o#' in triple[1]:
        g.remove(triple)

In [16]:
for s, p, o in g:
    print(f'{s} -> {p} -> {o}')

http://jajohnst.si676.si.umich.edu/omeka-s/api/items/308 -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://purl.org/dc/terms/Agent
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/310 -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://purl.org/dc/terms/Agent
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/303 -> http://purl.org/dc/terms/title -> A Mere Title for an item created via the API
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/309 -> http://xmlns.com/foaf/0.1/name -> Herman Melville
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/309 -> http://purl.org/dc/terms/title -> Herman Melville item for RDF demo
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/310 -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://omeka.org/s/vocabs/o#Item
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/309 -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://omeka.org/s/vocabs/o#Item
http://jajohnst.si676.si.umich.edu/omeka-s/api/item

### Adding New Triples to the Graph

To create new triples for the graph, use the information
previously created and stored in the `newData` variable.
This was a small dictionary. The process will use the `rdflib`
functions `.triples()` and `.add()`.

The `.triples()` is used here to match the new triples to
the existing nodes. In this case, the function will look for
existing nodes with a string that exactly matches
an existing node `foaf:name` property by looking for a *predicate*
with the *foaf* property and an *object* that matches the string literal.

Then, for each matching triple, the `.add()` function
creates new triples with the matched nodes as the *subject*,
builtin schema properties using a construction like `SDO.property`,
then provides the new data as the triple *object*.

In [17]:
# Note: this will only work if the Keys are in the data already on the site,
# so the data must be uploaded and added first

# loop through the "newData" dictionary with an 'author_name' iterator
for author_name in newData: 
    # match existing triples, stored as 3-part tuples, that match the 'author_name'
    for s, p, o, in g.triples((None, FOAF.name, Literal(author_name))):
        # retrieve the data by matching on the name (here stored in the object part of the matched triple)
        birthDate = newData[o.value]['https://schema.org/birthDate']
        deathDate = newData[o.value]['https://schema.org/deathDate']
        deathPlace = newData[o.value]['https://schema.org/deathPlace']

        # add the new data to the matched node
        g.add((s, SDO.birthDate, Literal(birthDate)))
        g.add((s, SDO.deathDate, Literal(deathDate)))
        g.add((s, SDO.deathPlace, URIRef(deathPlace)))

To see how the graph changed, serialize the new graph:

In [18]:
ser2 = g.serialize(format='turtle')

with open('item-set-graph-2.ttl', 'w') as f:
    f.write(ser2)

## Loading the data back to Omeka

Now loading the data back into Omeka is possible with the API! 
This step will require the development (or reuse) of a few more helper functions
that will write information to accomplish:

- authentication
- write linked data ready for JSON-LD that can be understood by Omeka S
- using a `.put()` request with the data object (as json-ld); note that this pulls the full data, adds new information and returns it to Omeka S; in production, with larger datasets, you might choose to only update new information, which uses a `.patch()` request in the Omeka S API

In [19]:
# authenticate
def get_credentials(credential_file_path):
    '''Retrieve Omeka S Api credentials from another file. 
    That file must be a JSON file.'''

    with open(credential_file_path, 'r') as credentials:
        keys = json.load(credentials)
    
    return keys['key_identity'], keys['key_credential']

key_identity, key_credential = get_credentials('../collection-site-materials/omeka-credentials.json')

In [20]:
# set up the request to use the items action
endpoint = 'http://jajohnst.si676.si.umich.edu/omeka-s/api'
action = '/items'

parameters = {
    'key_credential': key_credential,
    'key_identity'  : key_identity
}

headers = {
    'Content-Type': 'application/json'
}

A few helper functions:

- `get_omeka_item()` gets an Omeka item's JSON-LD data using an item id, returns a JSON object
- `create_property_dict_for_omekajson()` creates a dictionary containing property elements in the format Omeka uses
- `create_property_list_for_omekajson()` a list that wraps a dict that contains the elements for an Omeka S item
- `post_omeka_item()` posts the information back to the Omeka site


In [21]:
# helper functions
def get_omeka_item(endpoint, action, item):
    r = requests.get(endpoint + action + '/' + item)
    print(r.url)
    if r.status_code == 200:
#        return r.json() # returns as requests json obj
        return json.loads(r.text) # returns python json obj

def create_property_dict_for_omekajson(attr_id, attr_label, value, dtype='literal', lang='en-us', uri_label=None):
    prop_dict = {}
    prop_dict['type'] = dtype
    prop_dict['property_id'] = 'auto'
    prop_dict['property_label'] = attr_label
    prop_dict['is_public'] = True
    prop_dict['@value'] = value
    if lang:
        prop_dict['@language'] = lang
    if uri_label:
        prop_dict['o:label'] = uri_label
    return prop_dict

def create_property_list_for_omekajson(attr_id, attr_label, value, dtype='literal', lang='en-us', uri_label=None):
    attr_id = []
    prop_dict = {}
    prop_dict['type'] = dtype
    prop_dict['property_id'] = 'auto'
    prop_dict['property_label'] = attr_label
    prop_dict['is_public'] = True
    prop_dict['@value'] = value
    if lang:
        prop_dict['@language'] = lang
    if uri_label:
        prop_dict['o:label'] = uri_label
    attr_id.append(prop_dict)
    return attr_id

def post_omeka_item(endpoint, action, item, parameters, headers, data):
    r = requests.put(endpoint + action + '/' + item, params=parameters, headers=headers, data=data)
    print('posting to...',r.url)
    print('response code',r.status_code)
    if r.status_code == 200:
        print('Done!')
    else:
        print(r.text)


In [22]:
current_item = get_omeka_item(endpoint, action, '310')

#print(current_item)
print(json.dumps(current_item, indent=2))

http://jajohnst.si676.si.umich.edu/omeka-s/api/items/310
{
  "@context": "http://jajohnst.si676.si.umich.edu/omeka-s/api-context",
  "@id": "http://jajohnst.si676.si.umich.edu/omeka-s/api/items/310",
  "@type": [
    "o:Item",
    "dcterms:Agent"
  ],
  "o:id": 310,
  "o:is_public": true,
  "o:owner": {
    "@id": "http://jajohnst.si676.si.umich.edu/omeka-s/api/users/1",
    "o:id": 1
  },
  "o:resource_class": {
    "@id": "http://jajohnst.si676.si.umich.edu/omeka-s/api/resource_classes/1",
    "o:id": 1
  },
  "o:resource_template": null,
  "o:thumbnail": null,
  "o:title": "Jane Austen item for RDF demo",
  "thumbnail_display_urls": {
    "large": null,
    "medium": null,
    "square": null
  },
  "o:created": {
    "@value": "2025-11-14T03:51:30+00:00",
    "@type": "http://www.w3.org/2001/XMLSchema#dateTime"
  },
  "o:modified": {
    "@value": "2025-11-15T19:22:09+00:00",
    "@type": "http://www.w3.org/2001/XMLSchema#dateTime"
  },
  "o:primary_media": null,
  "o:media": [],
  

In [23]:
# create property list, integer as literal, no language attribute
jane_birth_to_add = create_property_dict_for_omekajson('schema:birthDate','Birth Date','1775',lang=None)

print(type(jane_birth_to_add))
print(jane_birth_to_add)
print('as JSON:\n', json.dumps(jane_birth_to_add))

<class 'dict'>
{'type': 'literal', 'property_id': 'auto', 'property_label': 'Birth Date', 'is_public': True, '@value': '1775'}
as JSON:
 {"type": "literal", "property_id": "auto", "property_label": "Birth Date", "is_public": true, "@value": "1775"}


In [24]:
# create birthplace as uri, specifying the type, adding an Omeka label, and no language attribute
jane_birthplace_as_uri = create_property_dict_for_omekajson('schema:birthPlace','Birth Place','https://en.wikipedia.org/wiki/England',dtype='uri',lang=None,uri_label='England (link to Wikipedia)')
print(jane_birthplace_as_uri)

{'type': 'uri', 'property_id': 'auto', 'property_label': 'Birth Place', 'is_public': True, '@value': 'https://en.wikipedia.org/wiki/England', 'o:label': 'England (link to Wikipedia)'}


In [25]:
# create birthplace as a literal, provide simplified lang attribute
jane_birthplace_as_literal = create_property_dict_for_omekajson('schema:birthPlace','Birth Place','England', lang='en')
print(jane_birthplace_as_literal)

{'type': 'literal', 'property_id': 'auto', 'property_label': 'Birth Place', 'is_public': True, '@value': 'England', '@language': 'en'}


In [26]:
# create a description in english
jane_description = create_property_dict_for_omekajson('dcterms:description','Description','English novelist')

print(jane_description)

{'type': 'literal', 'property_id': 'auto', 'property_label': 'Description', 'is_public': True, '@value': 'English novelist', '@language': 'en-us'}


Now, add the updates to the item data. Although ultimately you are working to build a JSON object,
all of the work in python will be done with dictionaries.
In this case, the basic idea is to add the property dictionaries (just created)
to keys in the dictionary that are named with the property names, e.g. `schema:birthDate` for the Birth Date property from schema.org.

In [27]:
# update the item by adding the new property list
updated_item = current_item
# add the new data elements, which are dictionaries, directly into lists
updated_item['schema:birthDate'] = [jane_birth_to_add]
updated_item['dcterms:description'] = [jane_description]

# demonstrate adding multiple properties (dictionaries embedded in a list)
updated_item['schema:birthPlace'] = [jane_birthplace_as_uri, jane_birthplace_as_literal]

#print(json.dumps(updated_item, indent=2))

A more "pythonic" and shorter way to write the above is to use the `update` method,
which allows you to add multiple elements in a combined statement.
This is particularly useful for adding elements to dictionaries (which are going to become JSON outputs) since it is efficient, groups the updates together, and makes the code clearer.

In [28]:
updated_item = current_item
updated_item.update({
    'schema:birthDate': [jane_birth_to_add],
    'schema:birthPlace': [jane_birthplace_as_uri, jane_birthplace_as_literal],
    'dcterms:description': [jane_description]
})

print(json.dumps(updated_item, indent=2))

{
  "@context": "http://jajohnst.si676.si.umich.edu/omeka-s/api-context",
  "@id": "http://jajohnst.si676.si.umich.edu/omeka-s/api/items/310",
  "@type": [
    "o:Item",
    "dcterms:Agent"
  ],
  "o:id": 310,
  "o:is_public": true,
  "o:owner": {
    "@id": "http://jajohnst.si676.si.umich.edu/omeka-s/api/users/1",
    "o:id": 1
  },
  "o:resource_class": {
    "@id": "http://jajohnst.si676.si.umich.edu/omeka-s/api/resource_classes/1",
    "o:id": 1
  },
  "o:resource_template": null,
  "o:thumbnail": null,
  "o:title": "Jane Austen item for RDF demo",
  "thumbnail_display_urls": {
    "large": null,
    "medium": null,
    "square": null
  },
  "o:created": {
    "@value": "2025-11-14T03:51:30+00:00",
    "@type": "http://www.w3.org/2001/XMLSchema#dateTime"
  },
  "o:modified": {
    "@value": "2025-11-15T19:22:09+00:00",
    "@type": "http://www.w3.org/2001/XMLSchema#dateTime"
  },
  "o:primary_media": null,
  "o:media": [],
  "o:item_set": [
    {
      "@id": "http://jajohnst.si676

Finally, test the `post_omeka_item()` function to post the update back to Omeka S:

In [29]:
# post updated information to Omeka
post_omeka_item(endpoint, action, '310', parameters=parameters, headers=headers, data=json.dumps(updated_item))

posting to... http://jajohnst.si676.si.umich.edu/omeka-s/api/items/310?key_credential=uYUFyILIrCKOj7tuJ0x5gvmy4I62intx&key_identity=ajGwGXFVse2w9PGRhAn9xZzxVzRuk3sm
response code 200
Done!


NOTES:

- in the above, the `uri` import does not work. the literals do. why?
- the `label` does not import, the resulting properties have the schema.org property name

## Insert information from the data graph

Finally, insert the new data from the graph, retrieve items, add the new data, and post it back to your Omeka site.

First, use a similar construction to the one used to pull information from the `newData` dictionary.
Since this is already in the graph and each *subject* is
an Omeka URI, pull the node IDs there to map the graph nodes to the corresponding Omeka items. Then using a dictionary called `graph_updates` add any information not in the update dictionary, and store this for input.

In [30]:
# Extract data from graph for each author
graph_updates = {}

for author_name in newData:
    for s, p, o in g.triples((None, FOAF.name, Literal(author_name))):
        # Extract item ID from the subject URI
        item_id = str(s).split('/')[-1]
        
        # Initialize dictionary for this item if not exists
        if item_id not in graph_updates:
            graph_updates[item_id] = {}
        
        # Get all schema properties for this subject
        for pred, obj in g.predicate_objects(s):
            if 'schema.org' in str(pred):
                property_name = str(pred).split('/')[-1]
                graph_updates[item_id][property_name] = obj

print(graph_updates)

{'310': {'birthPlace': rdflib.term.Literal('England', lang='en'), 'birthDate': rdflib.term.Literal('1775', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), 'deathDate': rdflib.term.Literal('1817', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), 'deathPlace': rdflib.term.URIRef('https://en.wikipedia.org/wiki/England')}, '312': {'identifier': rdflib.term.Literal('Q239739'), 'birthDate': rdflib.term.Literal('1947', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), 'deathDate': rdflib.term.Literal('2006', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), 'deathPlace': rdflib.term.URIRef('https://en.wikipedia.org/wiki/Lake_Forest_Park,_Washington')}, '309': {'birthDate': rdflib.term.Literal('1819', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), 'deathDate': rdflib.term.Literal('1891', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), 'deathPla

## Post the data back to Omeka S

Now that you have the information you need to update the data, use the `set_list` to get each Omeka item, update it with the new data, and post the data back to Omeka S.

In [31]:
# Loop through set_list to update each item
for item_id in set_list:
    item_id_str = str(item_id)
    
    # Get current item
    current_item = get_omeka_item(endpoint, action, item_id_str)
    
    # Check if there are updates for this item
    if item_id_str in graph_updates:
        updates = graph_updates[item_id_str]
        update_dict = {}
        
        # Create property dictionaries for each update
        if 'birthDate' in updates:
            birthdate_prop = create_property_list_for_omekajson(
                'schema:birthDate', 'Birth Date', 
                str(updates['birthDate']), lang=None
            )
            update_dict['schema:birthDate'] = birthdate_prop
            
        if 'deathDate' in updates:
            deathdate_prop = create_property_list_for_omekajson(
                'schema:deathDate', 'Death Date',
                str(updates['deathDate']), lang=None
            )
            update_dict['schema:deathDate'] = deathdate_prop
            
        if 'birthPlace' in updates:
            birthplace_prop = create_property_list_for_omekajson(
                'schema:birthPlace', 'Birth Place',
                str(updates['birthPlace']), dtype='uri', lang=None
            )
            update_dict['schema:birthPlace'] = birthplace_prop
        
        # Update the item
        current_item.update(update_dict)
        
        # Post back to Omeka
        post_omeka_item(endpoint, action, item_id_str, 
                       parameters=parameters, headers=headers, 
                       data=json.dumps(current_item))
        
        print(f'posted update for {item_id_str}')

http://jajohnst.si676.si.umich.edu/omeka-s/api/items/303
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/306
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/308
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/309
posting to... http://jajohnst.si676.si.umich.edu/omeka-s/api/items/309?key_credential=uYUFyILIrCKOj7tuJ0x5gvmy4I62intx&key_identity=ajGwGXFVse2w9PGRhAn9xZzxVzRuk3sm
response code 200
Done!
posted update for 309
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/310
posting to... http://jajohnst.si676.si.umich.edu/omeka-s/api/items/310?key_credential=uYUFyILIrCKOj7tuJ0x5gvmy4I62intx&key_identity=ajGwGXFVse2w9PGRhAn9xZzxVzRuk3sm
response code 200
Done!
posted update for 310
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/312
posting to... http://jajohnst.si676.si.umich.edu/omeka-s/api/items/312?key_credential=uYUFyILIrCKOj7tuJ0x5gvmy4I62intx&key_identity=ajGwGXFVse2w9PGRhAn9xZzxVzRuk3sm
response code 200
Done!
posted update for 312


Looks good! The set list contained six item IDs, but there were only updates for three items.
This indicates that the code successfully filtered the list, only identified corresponding entities
that required updates, transformed the data from the graph into JSON-LD, and then
posted updates to Omeka S.

✅ ETL complete!